In [139]:
pip install bs4

Note: you may need to restart the kernel to use updated packages.


In [140]:
conda install -c conda-forge parse

Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [141]:
pip install lxml

Note: you may need to restart the kernel to use updated packages.


In [226]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
print("Imported!")
import lxml.html as lh
import folium

Imported!


In [208]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [209]:
extract_data = requests.get(url).text
data = BeautifulSoup(extract_data, 'lxml')

In [210]:
cols = ['Postalcode','Borough','Neighborhood']
TO = pd.DataFrame(columns = column_names)
content = data.find('div', class_='mw-parser-output')
Table = content.table.tbody
Postalcode = 0
Borough = 0
Neighborhood = 0

In [211]:
# Parsing through the data

for tr in table.find_all('tr'):
    i = 0
    for td in tr.find_all('td'):
        if i == 0:
            postcode = td.text
            i = i + 1
        elif i == 1:
            borough = td.text
            i = i + 1
        elif i == 2: 
            neighborhood = td.text.strip('\n').replace(']','')
    TO = TO.append({'Postalcode': Postalcode,'Borough': Borough,'Neighborhood': Neighborhood},ignore_index=True)

In [212]:
TO = TO[TO.Borough!='Not assigned']
TO = TO[TO.Borough!= 0]
TO.reset_index(drop = True, inplace = True)
i = 0


In [216]:
# Now to create the loop for 
for i in range(0,TO.shape[0]):
    if TO.iloc[i][2] == 'Not assigned':
        TO.iloc[i][2] = TO.iloc[i][1]
        i = i+1
        

df = TO.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()

In [222]:
# Dropping Nan values and Not Assigned values from the dataframe
df = df.dropna()
df = df[~df.Borough.str.contains("Not Assigned")]

In [228]:
def neighborhood(grouped):    
    return ', '.join(sorted(grouped['Neighborhood'].tolist()))
                    
grp = df.groupby(['Postalcode', 'Borough'])
df_final = grp.apply(neighborhood).reset_index(name='Neighborhood')

In [227]:
print(df_final.shape)

(180, 3)


In [225]:
df_final

,Postalcode,Borough,Neighborhood
0,M1A\n,Not assigned\n,Not assigned\n
1,M1B\n,Scarborough\n,"Malvern, Rouge"
2,M1C\n,Scarborough\n,"Rouge Hill, Port Union, Highland Creek"
3,M1E\n,Scarborough\n,"Guildwood, Morningside, West Hill"
4,M1G\n,Scarborough\n,Woburn
...,...,...,...
175,M9V\n,Etobicoke\n,"South Steeles, Silverstone, Humbergate, Jamest..."
176,M9W\n,Etobicoke\n,"Northwest, West Humber - Clairville"
177,M9X\n,Not assigned\n,Not assigned\n
178,M9Y\n,Not assigned\n,Not assigned\n


In [203]:
df_final.to_csv('TO.csv', index=False)